<a href="https://colab.research.google.com/github/DarianSawali/NLP-Week2-Text-Generation/blob/main/News_RAG_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentence-transformers faiss-cpu newspaper3k bs4

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 102.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.9 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=c1cf276ced4c4619bf940166341c0ce1c2cc842daee4f2207f967385dcbd7ab8
  Stored in directory: /root/.cache/pip/wheels/a5/91/9f/00d66475960891a64867914273fcaf78df6cb04d905b104a2a
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=79804189f1d8d65fc425cfbfaf8c5c3f9e0332d94ce7d72bca967e971e166117
  Stored i

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import torch
import faiss
import numpy as np

In [3]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
gpt2 = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
documents = [
    {
        "text": "Burnaby RCMP are investigating after a pedestrian was struck near Metrotown on Monday morning. Emergency crews responded around 7 a.m. and the victim was transported to hospital with non-life-threatening injuries.",
        "url": "https://www.cbc.ca/news/canada/british-columbia",
        "source": "CBC BC"
    },
    {
        "text": "TransLink says SkyTrain service on the Expo Line was delayed Tuesday afternoon due to a mechanical issue near Commercial–Broadway Station, causing significant congestion during the rush hour commute.",
        "url": "https://globalnews.ca/bc",
        "source": "Global News BC"
    },
    {
        "text": "Vancouver police say they responded to a break-and-enter in Kitsilano overnight, urging residents to secure windows and report suspicious behaviour as property crime rises in the area.",
        "url": "https://bc.ctvnews.ca",
        "source": "CTV Vancouver"
    },
    {
        "text": "The B.C. government announced a new housing investment plan aimed at expanding affordable rentals across Metro Vancouver, with construction expected to begin in early 2025.",
        "url": "https://news.gov.bc.ca",
        "source": "BC Gov News"
    }
]

In [6]:
corpus_embeddings = embedder.encode([d["text"] for d in documents], convert_to_tensor=False)
corpus_embeddings = np.array(corpus_embeddings).astype("float32")

In [7]:
index = faiss.IndexFlatL2(corpus_embeddings.shape[1])
index.add(corpus_embeddings)

In [8]:
query = "What happened in Burnaby today?"
query_emb = embedder.encode(query)

D, I = index.search(np.array([query_emb]).astype("float32"), k=2)

retrieved = [documents[i] for i in I[0]]

In [17]:
tokenizer.pad_token = tokenizer.eos_token

context = "\n\n".join([f"[{d['source']}] {d['text']}" for d in retrieved])

prompt = (
    "You are a summarization system.\n"
    "You MUST summarize ONLY the information in the articles below.\n"
    "If something is NOT stated in the text, do NOT invent it.\n"
    "Include the EXACT sources provided.\n"
    "Do NOT guess or fabricate.\n\n"
    "ARTICLES:\n"
    f"{context}\n\n"
    "SUMMARY:"
)

inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

output = gpt2.generate(
    **inputs,
    max_length=200,
    do_sample=False,
    repetition_penalty=1.2,
    pad_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(output[0], skip_special_tokens=True))

You are a summarization system.
You MUST summarize ONLY the information in the articles below.
If something is NOT stated in the text, do NOT invent it.
Include the EXACT sources provided.
Do NOT guess or fabricate.

ARTICLES:
[CBC BC] Burnaby RCMP are investigating after a pedestrian was struck near Metrotown on Monday morning. Emergency crews responded around 7 a.m. and the victim was transported to hospital with non-life-threatening injuries.

[CTV Vancouver] Vancouver police say they responded to a break-and-enter in Kitsilano overnight, urging residents to secure windows and report suspicious behaviour as property crime rises in the area.

SUMMARY:
The following sections of this article will be updated throughout time so that you can find out what's going down at your local scene if there is any additional info about an incident involving pedestrians (or other vehicles) within walking distance from one another


In [18]:
tokenizer.pad_token = tokenizer.eos_token

context_lines = []
for d in retrieved:
    context_lines.append(f"- [{d['source']}] {d['text']}")
context = "\n".join(context_lines)

prompt = f"""
Summarize the following local news in 2–3 sentences.
Use ONLY the information provided.
Do NOT add anything that is not stated.
Mention the sources in parentheses at the end.

NEWS:
{context}

Summary:
""".strip()

inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

output_ids = gpt2.generate(
    **inputs,
    max_new_tokens=80,
    do_sample=False,
    repetition_penalty=1.2,
    pad_token_id=tokenizer.eos_token_id,
)

generated_only = output_ids[0][inputs["input_ids"].shape[1]:]
print(tokenizer.decode(generated_only, skip_special_tokens=True))

 The Mounties have identified two people who were involved last night's incident involving an elderly woman walking her dog outside of their home when she got stuck inside one of several parked cars behind them while trying desperately to get out from under another vehicle before being hit by something moving through it (the car had been driven into). Police said there has also be some evidence suggesting someone may've tried running away but no
